In [1]:
#!g1.1
import json
import os
import torch
from tqdm import tqdm
import scipy.io.wavfile as wav

from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.tensorboard import SummaryWriter

import numpy as np
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import math

import torchaudio
from boltons.fileutils import iter_find_files
from collections import OrderedDict

In [2]:
#!g1.1
# %pip install pytorch_lightning --upgrade

In [3]:
#!g1.1
from models import ConvFeatureEncoder, SegmentsRepr, SegmentsEncoder, NegativeSampler, SegmentPredictor, FinModel, FinModel1
from utils import ConstrativeLoss, sample_negatives
# from trainer import Trainer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

In [4]:
#!g1.1
# import wandb

In [5]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")

In [6]:
#!g1.1
def spectral_size(wav_len):
    layers = [(10,5,0), (8,4,0), (4,2,0), (4,2,0), (4,2,0)]
    for kernel, stride, padding in layers:
        wav_len = math.floor((wav_len + 2*padding - 1*(kernel-1) - 1)/stride + 1)
    return wav_len

In [7]:
#!g1.1
class Dataset_test:
    
    def __init__(self, path, segment_path, manifest_path, edges_path = None, chars = 'segments_chars/', frames = 'secs/'):
        with open(manifest_path, 'r') as json_file:
            manifest = json.load(json_file)
        self.manifest = manifest
#         self.manifest = manifest[:1000]
        self.path = path
        self.segment_path = segment_path
        self.frames = os.path.join(chars, frames)
        self.edges_path = edges_path
    
    def __len__(self):
        return len(self.manifest)
    
    
    def spectral_size(self, wav_len):
        layers = [(10,5,0), (8,4,0), (4,2,0), (4,2,0), (4,2,0)]
        for kernel, stride, padding in layers:
            wav_len = math.floor((wav_len + 2*padding - 1*(kernel-1) - 1)/stride + 1)
        return wav_len
    
    def __getitem__(self, ind):
#         print(self.manifest[ind])
        
        # Загрузка аудио-сигнала
        
        audio_filepath = self.manifest[ind]['audio_filepath']
#         print(audio_filepath)
        audio_file = os.path.join(self.path, audio_filepath)
#         print(audio_file)
        sampling_rate, signal = wav.read(audio_file)
#         signal, sampling_rate = torchaudio.load(path)
        
        # Обрезать тишину
        filetext_id = self.manifest[ind]['id']+'.txt'
        silero_filepath = audio_filepath.replace('.wav', '.txt')
        silero_file = os.path.join(self.edges_path, silero_filepath).replace('files/', '')
#         print(silero_file)
        
        with open(silero_file, 'r', encoding="cp1251") as file:
            tt = file.read()
        
        start = []
        end = []
        for line in tt.split('\n'):
            if len(line) > 0:
                start.append(eval(line.split()[0]))
                end.append(eval(line.split()[1]))
        start = min(start)
        end = max(end)
#         print(start)
#         print(end)
        signal = signal[start:end]
        
        # Загрузка разметки
        filetext_id = self.manifest[ind]['id']+'.txt'
        segment_filepath = audio_filepath.replace('.wav', '.txt').replace(filetext_id, '')
        
        segment_file = os.path.join(os.path.join(os.path.join(self.segment_path, segment_filepath), self.frames), filetext_id).replace('files/', '')
#         print(segment_file)
        with open(segment_file, 'r', encoding="cp1251") as file:
            tt = file.read()
        
        boundaries = set()
        mm = [i for i in tt.split('\n') if len(i)>0]
        for i in mm:
            boundaries.add(eval(i.split()[0]))
            boundaries.add(eval(i.split()[1]))
        boundaries = sorted(list(boundaries))
        
        return {'audio_file':os.path.join(self.path, audio_filepath), 
                'segment_file':segment_file, 
                'id':filetext_id, 'sample': signal, 
                'length': len(signal), 
                'spectral_size': self.spectral_size(len(signal)),
                'boundaries': boundaries}
        
#         return {'id':filetext_id, 'sample': signal, 'length': len(signal), 'boundaries': boundaries}

In [8]:
#!g1.1
def collate_fn(samples):
    
    max_length = max([sample['length'] for sample in samples])
    boundaries = [sample['boundaries'] for sample in samples]
    spectral_sizes = [sample['spectral_size'] for sample in samples]
    samples1 = []
    lengths = []
    samplings = []
    attentions = []
    ids = []
    audio_files = []
    segment_files = []
    for sample in samples:
        to_add_l = max_length-sample['length']
        sample1 = list(sample['sample'])+[0]*to_add_l
        samples1.append(torch.Tensor(sample1).unsqueeze(0))
        lengths.append(sample['length'])
        ids.append(sample['id'])
        audio_files.append(sample['audio_file'])
        segment_files.append(sample['segment_file'])
        att_norm = torch.ones(size = (1, sample['length']))
        att_add = torch.zeros(size = (1, to_add_l))
        att = torch.cat([att_norm, att_add], dim = -1)
        attentions.append(att)
        
    batch = torch.cat(samples1)
    lengths = torch.Tensor(lengths)
    attention_mask = torch.cat(attentions, dim = 0)
    spectral_size = torch.Tensor(spectral_sizes)
    
    return dict(batch=batch, lengths=lengths, attention_mask=attention_mask, 
                boundaries=boundaries, ids=ids, 
                audio_files=audio_files, 
                segment_files=segment_files, 
                spectral_size=spectral_size)

# Загрузка Данных

In [9]:
#!g1.1
test_farfield_dataset = Dataset_test('test/', 'segments_edges_init_test/', 'manifest_test_farfield.json', 
                        edges_path = 'silero_edges_test',
                              chars = 'segments_chars/', frames = 'secs/')
test_crowd_dataset = Dataset_test('test/', 'segments_edges_init_test/', 'manifest_test_crowd.json', 
                        edges_path = 'silero_edges_test',
                              chars = 'segments_chars/', frames = 'secs/')
test_dataset = Dataset_test('test/', 'segments_edges_init_test/', 'manifest_test.json', 
                        edges_path = 'silero_edges_test',
                              chars = 'segments_chars/', frames = 'secs/')

test_farfield_loader = DataLoader(test_farfield_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
test_crowd_loader = DataLoader(test_crowd_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)

In [10]:
#!g1.1
path_results = 'save_results_path_compare_golos'

In [11]:
#!g1.1


# Segment Model

In [12]:
#!g1.1
model_path = 'golos_model_segment_r_val_acc_200_edges_train_10_hours_model_segment.ckpt'

In [13]:
#!g1.1
accumulate_grad_batches = 1
cfg = {'optimizer': "adam",
'momentum': 0.9,
'learning_rate': 0.0001*accumulate_grad_batches,
'lr_anneal_gamma': 1.0,
'lr_anneal_step': 1000,
# 'epochs': 500,
'grad_clip': 0.5,
'batch_size': 8,

'conv_args': {},
'mask_args': {"segment": "first", "add_one": False},
'segm_enc_args': {},
'segm_predictor_args': {},
'loss_args': {"n_negatives": 1, "loss_args": {"reduction": "mean"}},
'num_epoch': 2}

In [14]:
#!g1.1
class Conf:
    def __init__(self, my_dict):
        for key, value in my_dict.items():
            setattr(self, key, value)
            
config = Conf(cfg)

In [15]:
#!g1.1
model = FinModel(config)
checkpoint = torch.load(model_path)
#     checkpoint = torch.load(model_path, map_location='cpu')

state_dicts = OrderedDict()
for key, value in checkpoint['state_dict'].items():
    state_dicts[key.replace('wav2vec_segm.', '')] = value
model.load_state_dict(state_dicts)

model.eval()
model=model.to('cuda')
name_path = 'model_segment'
names = ['farfield', 'crowd', 'all']
loaders = [test_farfield_loader, test_crowd_loader, test_loader]
fin_path = os.path.join(path_results, name_path)

for name, loader in zip(names, loaders):
    for batch in tqdm(loader):
        x = batch['batch']
        lengths = batch['lengths']
        attention_mask = batch['attention_mask']
        secs = batch['boundaries']
        ids = batch['ids']
        spectral_sizes = batch['spectral_size']
        os.makedirs(os.path.join(fin_path, name), exist_ok=True) 


        rr = model.compute_all(x.to('cuda'), secs, num_epoch=0, attention_mask=attention_mask.to('cuda'), return_secs=True)
    #         rr = model.compute_all(x, secs, num_epoch=0, attention_mask=attention_mask, return_secs=True)
        secs_preds = rr[1]['secs_pred']
        for idd, secs in zip(ids, secs_preds):
            with open(os.path.join(os.path.join(path_results, name_path), idd), 'w', encoding="cp1251") as file:
                file.write(str(secs))

100%|██████████| 1476/1476 [03:59<00:00,  6.16it/s]


In [16]:
#!g1.1


# Peak Detection Model

In [17]:
#!g1.1
model_path = 'golos_model_segment_r_val_acc_200_edges_train_10_hours_peak_detection_saved.ckpt'

In [21]:
#!g1.1
accumulate_grad_batches = 1
cfg = {'optimizer': "adam",
'momentum': 0.9,
'learning_rate': 0.0001*accumulate_grad_batches,
'lr_anneal_gamma': 1.0,
'lr_anneal_step': 1000,
# 'epochs': 500,
'grad_clip': 0.5,
'batch_size': 8,

'conv_args': {},
'mask_args': {"segment": "first", "add_one": False},
'segm_enc_args': {},
'segm_predictor_args': {},
'loss_args': {"n_negatives": 1, "loss_args": {"reduction": "mean"}},
'num_epoch': 2,
'use_projection': False}

In [22]:
#!g1.1
class Conf:
    def __init__(self, my_dict):
        for key, value in my_dict.items():
            setattr(self, key, value)
            
config = Conf(cfg)

In [23]:
#!g1.1
model = FinModel1(config)
checkpoint = torch.load(model_path)
#     checkpoint = torch.load(model_path, map_location='cpu')

state_dicts = OrderedDict()
for key, value in checkpoint['state_dict'].items():
    state_dicts[key.replace('wav2vec_segm.', '')] = value
model.load_state_dict(state_dicts)

model.eval()
model=model.to('cuda')
name_path = 'peak_detection'

names = ['farfield', 'crowd', 'all']
loaders = [test_farfield_loader, test_crowd_loader, test_loader]
fin_path = os.path.join(path_results, name_path)


for name, loader in zip(names, loaders):
    for batch in tqdm(loader):
        x = batch['batch']
        lengths = batch['lengths']
        attention_mask = batch['attention_mask']
        secs = batch['boundaries']
        ids = batch['ids']
        spectral_sizes = batch['spectral_size']
        os.makedirs(os.path.join(fin_path, name), exist_ok=True) 


        rr = model.compute_all(x.to('cuda'), secs, num_epoch=0, 
                               spectral_size = spectral_sizes,
                               attention_mask=attention_mask.to('cuda'), return_secs=True)
    #         rr = model.compute_all(x, secs, num_epoch=0, 
#                                spectral_size = spectral_sizes,
#         attention_mask=attention_mask, return_secs=True)

        secs_preds = rr[1]['secs_pred']
        for idd, secs in zip(ids, secs_preds):
            with open(os.path.join(os.path.join(path_results, name_path), idd), 'w', encoding="cp1251") as file:
                file.write(str(list(secs)))

100%|██████████| 1476/1476 [03:42<00:00,  6.62it/s]


In [24]:
#!g1.1


# Wav2Vec2 Model

In [34]:
#!g1.1
from modeling_segmentation import Wav2Vec2ModelForSegmentation

In [35]:
#!g1.1
model_path = 'golos_model_segment_r_val_transformers_acc_10_ep_500_r_val_edges_train_10_hours_new_loss.ckpt'

In [36]:
#!g1.1

model = Wav2Vec2ModelForSegmentation.from_pretrained("facebook/wav2vec2-base-960h")
checkpoint = torch.load(model_path)
#     checkpoint = torch.load(model_path, map_location='cpu')

state_dicts = OrderedDict()
for key, value in checkpoint['state_dict'].items():
    if 'project_back' not in key:
        state_dicts[key.replace('wav2vec_segm.', '')] = value
model.load_state_dict(state_dicts)

model.eval()
model=model.to('cuda')
name_path = 'wav2vec_model'

names = ['farfield', 'crowd', 'all']
loaders = [test_farfield_loader, test_crowd_loader, test_loader]
fin_path = os.path.join(path_results, name_path)


for name, loader in zip(names, loaders):
    for batch in tqdm(loader):
        x = batch['batch']
        lengths = batch['lengths']
        attention_mask = batch['attention_mask']
        secs = batch['boundaries']
        ids = batch['ids']
        spectral_sizes = batch['spectral_size']
        os.makedirs(os.path.join(fin_path, name), exist_ok=True) 

        rr = model.compute_all(x.to('cuda'), secs, num_epoch=0, attention_mask=attention_mask.to('cuda'), return_secs=True)
    #         rr = model.compute_all(x, secs, num_epoch=0, attention_mask=attention_mask, return_secs=True)
        secs_preds = rr[1]['secs_pred']
        for idd, secs in zip(ids, secs_preds):
            with open(os.path.join(os.path.join(path_results, name_path), idd), 'w', encoding="cp1251") as file:
                file.write(str(secs))

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ModelForSegmentation: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ModelForSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ModelForSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ModelForSegmentation were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1476/1476 [07:59<00:00,  3.08it/s]


In [ ]:
#!g1.1


# Агрегация данных

In [37]:
#!g1.1
import pandas as pd

In [47]:
#!g1.1
# Данная функция основана на https://github.com/felixkreuk/UnsupSeg/blob/master/utils.py

class RMetrics1(nn.Module):
    def __init__(self, eps = 1e-5, tolerance = 2, sampling_rate = 16000):
        super(RMetrics1, self).__init__()
        self.tolerance = tolerance
        self.eps = eps
        self.sampling_rate = sampling_rate
    
    def calculate_stride(self, isz, conv_layers):
        pad = 0
        insize = isz
        totstride = 1
        sec_per_frame = 1/self.sampling_rate

        for layer in conv_layers:
            kernel, stride = layer
            outsize = (insize + 2*pad - 1*(kernel-1)-1) / stride + 1
            insize = outsize
            totstride = totstride * stride

        RFsize = isz - (outsize - 1) * totstride

        ms_per_frame = sec_per_frame*RFsize*1000
        ms_stride = sec_per_frame*totstride*1000
        return outsize, totstride, RFsize, ms_per_frame, ms_stride
        
    def get_frames(self, secs, stride):
        frames = [[int(i*self.sampling_rate/stride) for i in sec] for sec in secs]
        return frames
        
    def make_true_boundaries(self, secs, boundaries, stride):
        frames = self.get_frames(secs, stride)
        true_boundaries = torch.zeros(size = boundaries.shape)
        for num_frame, frame in enumerate(frames):
            for i in frame:
                true_boundaries[num_frame, i] = 1
        return true_boundaries.long().detach().numpy()
    
    def get_sec_bounds(self, b, stride, attention_mask = None):
        if type(b)==torch.Tensor:
            b1 = b.long().detach().cpu().numpy()
        else:
            b1 = b
        
        if attention_mask is not None:
            b1 = b1*attention_mask.long().detach().cpu().numpy()
            
        frames_pred = []
        secs_pred = []
        for i in range(b1.shape[0]):
            frames = np.where(b1[i, :] == 1)[0]
            secs = [i*stride/self.sampling_rate for i in frames]
            frames_pred.append(frames)
            secs_pred.append(secs)
        return frames_pred, secs_pred
    
    def get_precision_recall_frames(self, true_boundaries, b, attention_mask = None):
        if type(b)==torch.Tensor:
            b1 = b.long().detach().numpy()
        else:
            b1 = b
            
        if attention_mask is not None:
            b1 = b1*attention_mask.long().detach().cpu().numpy()
            
        recall = recall_score(true_boundaries.flatten(), b1.flatten())
        pre = precision_score(true_boundaries.flatten(), b1.flatten())
        f_score = f1_score(true_boundaries.flatten(), b1.flatten())
        return recall, pre, f_score
    
    def get_stats(self, frames_true, frames_pred):
        
        # Утащено отсюда: https://github.com/felixkreuk/UnsupSeg/blob/68c2c7b9bd49f3fb8f51c5c2f4d5aa85f251eaa8/utils.py#L69
        precision_counter = 0 
        recall_counter = 0
        pred_counter = 0 
        gt_counter = 0

        for (y, yhat) in zip(frames_true, frames_pred):
            for yhat_i in yhat:
                min_dist = np.abs(np.array(y) - yhat_i).min()
                precision_counter += (min_dist <= self.tolerance)
            for y_i in y:
                min_dist = np.abs(np.array(yhat) - y_i).min()
                recall_counter += (min_dist <= self.tolerance)
            pred_counter += len(yhat)
            gt_counter += len(y)

        return precision_counter, recall_counter, pred_counter, gt_counter
    
    def calc_metr(self, precision_counter, recall_counter, pred_counter, gt_counter):

        # Утащено отсюда: https://github.com/felixkreuk/UnsupSeg/blob/68c2c7b9bd49f3fb8f51c5c2f4d5aa85f251eaa8/utils.py#L69
        EPS = 1e-7

        precision = precision_counter / (pred_counter + self.eps)
        recall = recall_counter / (gt_counter + self.eps)
        f1 = 2 * (precision * recall) / (precision + recall + self.eps)

        os = recall / (precision + EPS) - 1
        r1 = np.sqrt((1 - recall) ** 2 + os ** 2)
        r2 = (-os + recall - 1) / (np.sqrt(2))
        rval = 1 - (np.abs(r1) + np.abs(r2)) / 2

        return precision, recall, f1, rval
    
    def get_metrics(self, true_secs, b, seq_len, config, attention_mask = None, 
                    return_secs=False):
        
        outsize, totstride, RFsize, ms_per_frame, ms_stride = self.calculate_stride(seq_len, config)
#         print(seq_len, outsize, totstride, RFsize, ms_per_frame, ms_stride)
        frames_true = self.get_frames(true_secs, totstride)
        frames_pred, secs_pred = self.get_sec_bounds(b, totstride, attention_mask = attention_mask)
        precision_counter, recall_counter, pred_counter, gt_counter = self.get_stats(frames_true, frames_pred)
        precision, recall, f1, rval = self.calc_metr(precision_counter, recall_counter, pred_counter, gt_counter)
        if return_secs:
            return precision, recall, f1, rval, secs_pred
        else:
            return precision, recall, f1, rval
        
    def get_metrics_secs(self, true_secs, secs_pred, totstride = 160):
        
        frames_true = self.get_frames(true_secs, totstride)
        frames_pred = self.get_frames(secs_pred, totstride)
        precision_counter, recall_counter, pred_counter, gt_counter = self.get_stats(frames_true, frames_pred)
        precision, recall, f1, rval = self.calc_metr(precision_counter, recall_counter, pred_counter, gt_counter)
        return precision, recall, f1, rval
    
    def get_metrics_secs1(self, true_secs, secs_pred, totstride = 160):
        
#         frames_true = self.get_frames(true_secs, totstride)
#         frames_pred = self.get_frames(secs_pred, totstride)
        precision_counter, recall_counter, pred_counter, gt_counter = self.get_stats(true_secs, secs_pred)
        precision, recall, f1, rval = self.calc_metr(precision_counter, recall_counter, pred_counter, gt_counter)
        return precision, recall, f1, rval
    
    

In [48]:
#!g1.1
def read_true_file(segment_file):
    with open(segment_file, 'r', encoding="cp1251") as file:
        tt = file.read()

    boundaries = set()
    mm = [i for i in tt.split('\n') if len(i)>0]
    for i in mm:
        boundaries.add(eval(i.split()[0]))
        boundaries.add(eval(i.split()[1]))
    boundaries = sorted(list(boundaries))
    return boundaries

In [49]:
#!g1.1
def read_pred_file(segment_file):
    with open(segment_file, 'r', encoding="cp1251") as file:
        tt = file.read()
    boundaries = eval(tt)
    return boundaries

In [50]:
#!g1.1
def collate_fn_res(samples):

    boundaries = [sample['boundaries'] for sample in samples]
    samples1 = []
    lengths = []
    samplings = []
    attentions = []
    ids = []
    audio_files = []
    segment_files = []
    for sample in samples:
        ids.append(sample['id'])
        audio_files.append(sample['audio_file'])
        segment_files.append(sample['segment_file'])
        
    
    return dict(batch=batch, lengths=lengths, 
                boundaries=boundaries, ids=ids, 
                audio_files=audio_files, 
                segment_files=segment_files)

In [56]:
#!g1.1
test_farfield_dataset = Dataset_test('test/', 'segments_edges_init_test/', 'manifest_test_farfield.json', 
                        edges_path = 'silero_edges_test',
                              chars = 'segments_chars/', frames = 'secs/')
test_crowd_dataset = Dataset_test('test/', 'segments_edges_init_test/', 'manifest_test_crowd.json', 
                        edges_path = 'silero_edges_test',
                              chars = 'segments_chars/', frames = 'secs/')
test_dataset = Dataset_test('test/', 'segments_edges_init_test/', 'manifest_test.json', 
                        edges_path = 'silero_edges_test',
                              chars = 'segments_chars/', frames = 'secs/')

test_farfield_loader = DataLoader(test_farfield_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
test_crowd_loader = DataLoader(test_crowd_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=8, collate_fn = collate_fn)

In [57]:
#!g1.1


In [58]:
#!g1.1
path_results = 'save_results_path_compare_golos'

folders = ['model_segment', 'peak_detection', 'wav2vec_model']
names = ['farfield', 'crowd', 'all']
loaders = [test_farfield_loader, test_crowd_loader, test_loader]
datasets = [test_farfield_dataset, test_crowd_dataset, test_dataset]

In [59]:
#!g1.1


In [63]:
#!g1.1
result_dataframes = []
metr = RMetrics1(tolerance=2)
totstride = 160

for folder in folders:
    path_folder = os.path.join(path_results, folder)
    for name, dataset in zip(names, datasets):
#         name_path = os.path.join(path_folder, name)
        idss = os.listdir(path_folder)

        secs_preds = []
        secs_trues = []

        for num in tqdm(range(len(dataset))):

            idd = dataset[num]
            true_file = idd['segment_file']
            bound_true = idd['boundaries']
            ids = idd['id']

    #         bound_true = read_true_file(true_files[num])
            bound_pred = read_pred_file(os.path.join(path_folder, ids))

            secs_trues.append(bound_true)
            secs_preds.append(bound_pred)

        precision, recall, f1, rval = metr.get_metrics_secs(secs_trues, secs_preds, totstride = totstride)

        datafr = pd.DataFrame([folder, name, 
                               precision, recall, 
                               f1, rval]).T.rename(columns = {0:'type', 
                                                              1:'name',
                                                              2:'precision',
                                                              3:'recall', 
                                                              4:'f1', 
                                                              5:'rval'})

        result_dataframes.append(datafr)

100%|██████████| 11804/11804 [00:55<00:00, 214.41it/s]


In [ ]:
#!g1.1


In [64]:
#!g1.1
result_df = pd.concat(result_dataframes, ignore_index = True)

In [65]:
#!g1.1
result_df

,type,name,precision,recall,f1,rval
0,model_segment,farfield,0.569363,0.581862,0.57554,0.635048
1,model_segment,crowd,0.562237,0.581969,0.571928,0.630045
2,model_segment,all,0.563326,0.581952,0.572482,0.630831
3,peak_detection,farfield,0.562177,0.818671,0.666597,0.5291
4,peak_detection,crowd,0.555358,0.565633,0.560443,0.622506
5,peak_detection,all,0.556949,0.605537,0.580223,0.627695
6,wav2vec_model,farfield,0.661687,0.386808,0.488211,0.559747
7,wav2vec_model,crowd,0.574688,0.48842,0.528049,0.605628
8,wav2vec_model,all,0.584543,0.472396,0.522515,0.600593


In [66]:
#!g1.1
result_df.to_csv('results_compare_golos.csv')

In [ ]:
#!g1.1
